In [1]:
%%file polint.f90
SUBROUTINE polint(xa, ya, n, x, y, dy)
INTEGER, PARAMETER :: NMAX = 10
INTEGER :: n,i, m, ns
REAL*8, intent(in) :: x
REAL*8, intent(out) :: y, dy
REAL*8, intent(in), DIMENSION(n) :: xa, ya
REAL*8, DIMENSION(NMAX) :: c, d
REAL*8 :: den, dif, dift, ho, hp, w

!f2py intent(in) xa,ya,x
!f2py intent(out) y,dy
!f2py intent(hide) c,d


! Initialize
ns = 1
dif = ABS(x - xa(1))

! Find the closest point
DO i = 1, n
dift = ABS(x - xa(i))
IF (dift < dif) THEN
        ns = i
        dif = dift
END IF
c(i) = ya(i)
d(i) = ya(i)
END DO

y = ya(ns)
ns = ns - 1

! Perform the interpolation
DO m = 1, n-1
DO i = 1, n-m
        ho = xa(i) - x
        hp = xa(i+m) - x
        w = c(i+1) - d(i)
        den = ho - hp
        IF (den == 0.0) THEN
        PRINT *, 'Failure in polint: division by zero'
        STOP
        END IF
        den = w / den
        d(i) = hp * den
        c(i) = ho * den
END DO
IF (2 * ns < n - m) THEN
        dy = c(ns + 1)
ELSE
        dy = d(ns)
        ns = ns - 1
END IF
y = y + dy
END DO

RETURN
END subroutine polint

Overwriting polint.f90


In [1]:
import os, sys

In [3]:
!{sys.executable} -m numpy.f2py --quiet -c polint.f90 -m mpolint

Could not locate executable g77
Could not locate executable f77
Could not locate executable ifort
Could not locate executable ifl
Could not locate executable f90
Could not locate executable DF
Could not locate executable efl


d:\Anaconda\lib\site-packages\setuptools\_distutils\cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
Using built-in specs.
COLLECT_GCC=C:\msys64\ucrt64\bin\gfortran.exe
COLLECT_LTO_WRAPPER=C:/msys64/ucrt64/bin/../lib/gcc/x86_64-w64-mingw32/14.2.0/lto-wrapper.exe
Target: x86_64-w64-mingw32
Configured with: ../gcc-14.2.0/configure --prefix=/ucrt64 --with-local-prefix=/ucrt64/local --build=x86_64-w64-mingw32 --host=x86_64-w64-mingw32 --target=x86_64-w64-mingw32 --with-native-system-header-dir=/ucrt64/include --libexecdir=/ucrt6

In [2]:
%pwd

'd:\\Computation methods'

In [6]:
# script to test the polint module
from scipy import *
from matplotlib.pyplot import *
import mpolint as p
from numpy import sin,linspace,array # sin, linspace and
# array used to be part of scipy but have been shifted to
# numpy
from numpy import pi
# interpolate on a table of sin(x) as a test
xx=linspace(0,2*pi,10) #
# create
yy=xx*sin(xx)
x=linspace(-pi,3*pi,501) # points at which to interpolate
y=[p.polint(xx,yy,w)[0] for w in x]
dy=[p.polint(xx,yy,w)[1] for w in x]
# plot the outputs
figure(1)
plot(xx,yy,"ro",x,y,"r",x,x*sin(x),"g")
title("Interpolating sin(x)")
legend(["Table values","Interpolated values","True function"])
figure(2)
1
semilogy(x,abs(array(dy)),"r",x,abs(array(x*sin(x)-y)),"g")
title("Estimated")
legend(["Estimated Error","True Error"])
show()
p.polint

ImportError: DLL load failed while importing mpolint: The specified module could not be found.

In [1]:
import numpy as np
import sympy as sp

def neville_algorithm(x, y):
    """
    Neville's Algorithm for polynomial interpolation with printing of polynomial expressions at each step.
    
    Args:
    x : array-like
        The x-coordinates of the known points.
    y : array-like
        The y-coordinates of the known points.
    
    Returns:
    None
    """
    n = len(x)
    # Create a matrix to store polynomial expressions
    P = np.empty((n, n), dtype=object)

    # Define the symbol for the variable
    t = sp.symbols('t')

    # Initialize the first column of P with y-values
    for i in range(n):
        P[i, 0] = y[i]
    
    # Compute the polynomial at each step
    for j in range(1, n):
        for i in range(n - j):
            P[i, j] = ((t - x[i]) * P[i + 1, j - 1] - (t - x[i + j]) * P[i, j - 1]) / (x[i + j] - x[i])
    
    # Print the polynomial matrix
    print("Polynomial Matrix:")
    header = ["Step 0"] + [f"Step {i}" for i in range(1, n)]
    print(f"{'':<10}", end="")
    for h in header:
        print(f"{h:<60}", end="")
    print()
    
    for i in range(n):
        print(f"P[{i}, :]:", end="")
        for j in range(n):
            expr = P[i, j]
            if expr is not None:
                # Convert the expression to a string
                expr_str = sp.pretty(expr, use_unicode=True)
                print(f"{expr_str:<60}", end="")
            else:
                print(f"{'':<60}", end="")
        print()
    
    # Print polynomial expressions for each step
    for j in range(1, n):
        polynomial_expr = P[0, j]
        simplified_poly = sp.simplify(polynomial_expr)
        print(f"Step {j}: Polynomial expression:")
        # Convert the expression to a string
        polynomial_expr_str = sp.pretty(simplified_poly, use_unicode=True)
        print(polynomial_expr_str)
        print()

# Sample data points
x_sample = np.linspace(0, 1, 5)  # 5 equally spaced points from 0 to 1
y_sample = np.sin(x_sample) + x_sample**2  # Function values at sample points

# Run Neville's algorithm and print polynomial expressions
neville_algorithm(x_sample, y_sample)

Polynomial Matrix:
          Step 0                                                      Step 1                                                      Step 2                                                      Step 3                                                      Step 4                                                      
P[0, :]:0.0                                                         1.23961583701809⋅t                                          -2.47923167403618⋅t⋅(t - 0.5) + 2.0⋅t⋅(1.67808631739872⋅t - 0.109617620095157)1.33333333333333⋅t⋅(-2.0⋅(t - 0.75)⋅(1.67808631739872⋅t - 0.109617620095157) +
 2.0⋅(t - 0.25)⋅(2.05885288567652⋅t - 0.300000904234059)) - 1.33333333333333⋅(
t - 0.75)⋅(-2.47923167403618⋅t⋅(t - 0.5) + 2.0⋅t⋅(1.67808631739872⋅t - 0.10961
7620095157))1.0⋅t⋅(-1.33333333333333⋅(t - 1.0)⋅(-2.0⋅(t - 0.75)⋅(1.67808631739872⋅t - 0.10
9617620095157) + 2.0⋅(t - 0.25)⋅(2.05885288567652⋅t - 0.300000904234059)) + 1.
33333333333333⋅(t - 0.25)⋅(-2.0⋅(t - 1.0)⋅(2.058852885676